Import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SequentialFeatureSelector, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [ ]:
# Load the diabetes dataset
diabetes_data = pd.read_csv('diabetes.csv')

# 1. Missing Values Ratio: Remove columns with more than 30% missing values
diabetes_data = diabetes_data.loc[:, diabetes_data.isnull().mean() < 0.3]

In [ ]:
# Impute remaining missing values with the median
imputer = SimpleImputer(strategy='median')
diabetes_data = pd.DataFrame(imputer.fit_transform(diabetes_data), columns=diabetes_data.columns)

# Separate features and target
X_diabetes = diabetes_data.drop('Outcome', axis=1)
y_diabetes = diabetes_data['Outcome']

# Split data into training and testing sets
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_diabetes, y_diabetes, test_size=0.2, random_state=42)

In [ ]:
# 2. High Correlation Filter: Remove highly correlated features (correlation > 0.8)
def remove_highly_correlated_features(data, threshold=0.8):
    corr_matrix = data.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    return data.drop(to_drop, axis=1)

In [ ]:
X_train_d = remove_highly_correlated_features(X_train_d, threshold=0.8)
X_test_d = X_test_d[X_train_d.columns]

# 3. Low Variance Filter
variance_filter_d = VarianceThreshold(threshold=0.1)
X_train_d_var = variance_filter_d.fit_transform(X_train_d)
X_test_d_var = variance_filter_d.transform(X_test_d)

In [ ]:
# Train RandomForest model after Low Variance Filter
model_d_var = RandomForestClassifier(n_estimators=100, random_state=42)
model_d_var.fit(X_train_d_var, y_train_d)
y_pred_d_var = model_d_var.predict(X_test_d_var)
print(f'Accuracy for Diabetes after Low Variance Filter: {accuracy_score(y_test_d, y_pred_d_var)}')

# 4. Forward Feature Selection
model_d_fwd = LogisticRegression()
sfs_d = SequentialFeatureSelector(model_d_fwd, n_features_to_select=5, direction='forward')
sfs_d.fit(X_train_d, y_train_d)

# Transform datasets
X_train_d_fwd = sfs_d.transform(X_train_d)
X_test_d_fwd = sfs_d.transform(X_test_d)

Accuracy for Diabetes after Low Variance Filter: 0.7207792207792207


In [ ]:
# Train and evaluate model after Forward Feature Selection
model_d_fwd.fit(X_train_d_fwd, y_train_d)
y_pred_d_fwd = model_d_fwd.predict(X_test_d_fwd)
print(f'Accuracy for Diabetes after Forward Feature Selection: {accuracy_score(y_test_d, y_pred_d_fwd)}')

# 5. Backward Feature Elimination
model_d_bwd = DecisionTreeClassifier()
rfe_d = RFE(model_d_bwd, n_features_to_select=5)
rfe_d.fit(X_train_d, y_train_d)

# Transform datasets
X_train_d_bwd = rfe_d.transform(X_train_d)
X_test_d_bwd = rfe_d.transform(X_test_d)

# Train and evaluate model after Backward Feature Elimination
model_d_bwd.fit(X_train_d_bwd, y_train_d)
y_pred_d_bwd = model_d_bwd.predict(X_test_d_bwd)
print(f'Accuracy for Diabetes after Backward Feature Elimination: {accuracy_score(y_test_d, y_pred_d_bwd)}')

Accuracy for Diabetes after Forward Feature Selection: 0.7727272727272727
Accuracy for Diabetes after Backward Feature Elimination: 0.6883116883116883


In [ ]:
# 6. Random Forest Feature Importance: Keep only top 5 most important features
model_d_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_d_rf.fit(X_train_d, y_train_d)
importance_d = pd.Series(model_d_rf.feature_importances_, index=X_train_d.columns)
top_features_d = importance_d.nlargest(5).index

# Train and evaluate model using only top 5 features
X_train_d_rf = X_train_d[top_features_d]
X_test_d_rf = X_test_d[top_features_d]
model_d_rf.fit(X_train_d_rf, y_train_d)
y_pred_d_rf = model_d_rf.predict(X_test_d_rf)
print(f'Accuracy for Diabetes with Top 5 Random Forest Features: {accuracy_score(y_test_d, y_pred_d_rf)}')

Accuracy for Diabetes with Top 5 Random Forest Features: 0.7792207792207793
